### Automatically detect and model the background emission using LUCI
In this notebook, we will demonstrate the tools in LUCI to detect and model the background.

To do this, we apply the following steps:
1) Use a segmentation algorithm to find the background pixels
2) Use PCA to construct a subspace representing the background component
    - This includes the sky lines
3) Project each background pixel into a truncated PCA space
4) Interpolate masked pixels into PCA space
    - This can be done by either standard interpolation or using an artificial neural network

All of these steps have been wrapped into a single LUCI call for convenience. Implementation details can be found in our article.

In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from astroML.correlation import two_point
plt.style.use('/home/carterrhea/Documents/LUCI/light.mplstyle')

In [4]:
# Define user-specific variables
n_components = 50  # Number of PCA components to calculate
n_components_keep = 3  # Number of PCA components to keep
outputDir = '/export/home/carterrhea/NGC4449'  # Output directory for plots and data
Luci_path = '/home/carterrhea/Documents/LUCI/'
cube_dir = '/export/home/carterrhea/NGC4449'  # Path to data cube
cube_name = 'NGC4449_SN3'  # don't add .hdf5 extension
object_name = 'NGC4449'
redshift = 0.0004  # Redshift
resolution = 5000

path = os.path.abspath(Luci_path)
sys.path.insert(0, path)  # add LUCI to the available paths
plt.style.use(os.path.join(Luci_path, 'light.mplstyle'))
from LuciBase import Luci

2023-08-30 14:49:00.057864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 14:49:00.227085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/src/heasoft-6.31.1/x86_64-pc-linux-gnu-libc2.35/lib
2023-08-30 14:49:00.227118: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-30 14:49:01.990263: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror:

In [5]:
cube = Luci(Luci_path, cube_dir+'/'+cube_name, cube_dir, object_name, redshift, resolution, mdn=False)


Reading in data...


100%|█████████████████████████████████████████████| 9/9 [00:46<00:00,  5.17s/it]


In [6]:
BkgTransformedPCA, pca, interpolatedSourcePixels, idx_bkg, idx_src, coefficient_array = cube.create_background_subspace(x_min=500, x_max=1500, y_min=500, y_max=1500, 
                                                                                                     n_components=n_components, n_components_keep=n_components_keep)

Existing deep frame extracted from hdf5 file.
Epoch 1/100


2023-08-30 14:51:17.259157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/src/heasoft-6.31.1/x86_64-pc-linux-gnu-libc2.35/lib
2023-08-30 14:51:17.259229: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-30 14:51:17.259284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dubble): /proc/driver/nvidia/version does not exist
2023-08-30 14:51:17.259536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fla

65130/65138 [============================>.] - ETA: 0s - loss: 0.1697 - mae: 0.4629 - mape: 1399.2238

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f951a8c23a0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
import pickle
pickle.dump(idx_bkg, open('/export/home/carterrhea/NGC4449/idx_bkg.pkl', 'wb'))
pickle.dump(idx_src, open('/export/home/carterrhea/NGC4449/idx_src.pkl', 'wb'))
pickle.dump(cube.spectrum_axis, open('/export/home/carterrhea/NGC4449/spectrum_axis.pkl', 'wb'))
pickle.dump(BkgTransformedPCA, open('/export/home/carterrhea/NGC4449/BkgTransformedPCA.pkl', 'wb'))

#pickle.dump(cube.cube_final, open('/export/home/carterrhea/NGC4449/cube_final.pkl','wb'))

In [ ]:
import numpy as np
min_spectral_scale = np.argmin(np.abs([1e7 / wavelength - 675 for wavelength in cube.spectrum_axis]))
max_spectral_scale = np.argmin(np.abs([1e7 / wavelength - 670 for wavelength in cube.spectrum_axis]))

spec_scale = np.nanmax(cube.cube_final[600, 700, min_spectral_scale:max_spectral_scale])
max_spectral = np.argmin(np.abs([1e7 / wavelength - 646 for wavelength in cube.spectrum_axis]))
min_spectral = np.argmin(np.abs([1e7 / wavelength - 678 for wavelength in cube.spectrum_axis]))

reconstructed_spectra = pca.mean_ + np.sum([pca.components_[i] * interpolatedSourcePixels[10][i] for i in range(1, n_components_keep)], axis=0)


In [ ]:
print(spec_scale)

In [ ]:
bkg_axis, bkg_sky = cube.extract_spectrum_region(cube_dir+'/bkg.reg', mean=True)

In [ ]:
fig, ax = plt.subplots()

#plt.plot(1e7/cube.spectrum_axis[min_spectral: max_spectral], pca.mean_-np.sum([pca.components_[i] * BkgTransformedPCA[0,i] for i in range(n_components_keep)], axis=0), label='Reconstructed Bkg 1')
#plt.plot(1e7/cube.spectrum_axis[min_spectral: max_spectral], pca.mean_, linewidth=3, `linestyle='-.', label='Mean')
plt.plot(1e8/cube.spectrum_axis[min_spectral: max_spectral],
         spec_scale*reconstructed_spectra,
         linewidth=3, linestyle='-', label='Reconstructed Source 1')
plt.plot(1e8/cube.spectrum_axis[min_spectral: max_spectral], bkg_sky[min_spectral: max_spectral], label='Standard Background', linestyle='--')
plt.vlines((10*654.95*(1+redshift), 10*656.3*(1+redshift), 10*658.35*(1+redshift), 10*671.7*(1+redshift), 10*673.1*(1+redshift)), -0.1*spec_scale, 0.65*spec_scale, linestyle='--', color='grey')
ax.text(s=r'[NII]6548', x=10*654.8*(1+redshift)-15, y=0.67*spec_scale, fontsize='large', fontweight='heavy') 
ax.text(s=r'H$\alpha$', x=10*656.3*(1+redshift)-4.5, y=0.69*spec_scale, fontsize='large', fontweight='heavy') 
ax.text(s=r'[NII]6584', x=10*658.3*(1+redshift)-15, y=0.67*spec_scale, fontsize='large', fontweight='heavy') 
ax.text(s=r'[SII]6716,6731', x=10*671.7*(1+redshift)-18, y=0.67*spec_scale, fontsize='large', fontweight='heavy') 
#ax.text(s=r'[SII]6731', x=673.1*(1+redshift)-1.5, y=0.67*spec_scale, fontsize='large', fontweight='heavy') 


#plt.ylim(-0.1, 0.6)
plt.ylim(-0.1*spec_scale, 0.65*spec_scale)
plt.legend()
plt.xlabel('Wavelength (Å)', fontsize=18, fontweight='bold', fontname='Roboto')
plt.ylabel(r'Flux (ergs/s/cm$^{2}$/Å)', fontsize=18, fontweight='bold', fontname='Roboto')
ax.tick_params(labelsize=14)
plt.savefig('/export/home/carterrhea/NGC4449/Backgrounds.png')

In [ ]:
np.std([val[0] for val in interpolatedSourcePixels])

In [ ]:
plt.hist([val[0] for val in BkgTransformedPCA])

# Check if a component is noise
We can estimate whether or not a component is noise-dominated (i.e. not containing any useful signal) by calculating the *n*-point correlation function. If the value of the function for a spectrum is near zero, then the spectrum is pure noise.


In [ ]:
cube.fit_cube(['Halpha', 'NII6583', 'NII6548'], 
                 'sinc', 
                 [1,1,1], [1,1,1],
                 #950, 1100, 950, 1100, 
                  600, 1500, 500, 1500,
                 spec_min= 15000, spec_max = 15500,
                 bkgType='pca', n_threads=40, binning=2, bayes_bool=False, n_stoch=1,
                 pca_coefficient_array=coefficient_array, pca_vectors=pca.components_, pca_mean=pca.mean_
             )